In [ ]:
!pip3 install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 36.7 MB/s 
     |████████████████████████████████| 5.8 MB 53.4 MB/s 
     |████████████████████████████████| 132 kB 80.1 MB/s 
     |████████████████████████████████| 182 kB 92.5 MB/s 
     |████████████████████████████████| 212 kB 79.0 MB/s 
     |████████████████████████████████| 127 kB 80.2 MB/s 
     |████████████████████████████████| 7.6 MB 67.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
#import praw
import tqdm
import tensorflow_hub as hub
import tensorflow as tf
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

from sklearn.model_selection import train_test_split

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
from torch.nn import functional as F
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers.utils import logging

import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [ ]:
main_data=pd.read_csv("/content/drive/MyDrive/NLP Project/main_data.csv")

In [ ]:
main_data

,Comments,Reply
0,"Example: Reddit, we lost something. Can you he...",This is a good example of a case Reddit was ab...
1,Case: Scum of the Earth nan,"So was this actually solved by reddit, or is i..."
2,Some thoughts on how this sub should be run. I...,Does the sub plan on address witchhunts and th...
3,Some thoughts on how this sub should be run. I...,moderation at /r/askscience is excellent and i...
4,Some thoughts on how this sub should be run. I...,Are you purely working with any issue on reddi...
...,...,...
415946,I have not read all of the posts on this threa...,Here's another reason. They were able to ID t...
415947,This is like the 12th time I've seen this post...,It is a HUGE deal!
415948,Now find the murderer please. Or the murderer’...,Parents are both dead so it will likely be the...
415949,This is great news. Strange that he has siblin...,If his siblings are younger than him there’s a...


**Load the tokenizer and pretrained model**

In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

**Add special tokens** 

In [ ]:

special_tokens = {
    'bos_token': "<bos>",
    'additional_special_tokens': ["<Question>", "<Reply>"],
    'pad_token': "<pad>",
    'mask_token' : "<msk>"
}

num_new_tokens = tokenizer.add_special_tokens(special_tokens)
vocab = tokenizer.get_vocab()
model.resize_token_embeddings(len(vocab))

Embedding(50262, 768)

In [ ]:
tokenizer("<msk>")

{'input_ids': [50261], 'attention_mask': [1]}

In [ ]:
dialogues = main_data.values.tolist()   

**Change structure of data**

In [ ]:
dialogue_csv = pd.DataFrame()
bos_id = "<bos>"
speaker1_id = "<Question>"
speaker2_id = "<Reply>"
eos_id = "<|endoftext|>"
mask_id = "<msk>"
list_d = []
list_l = []
list_t = []



for i in tqdm.tqdm(range(len(dialogues))):
    curr_dial = str(bos_id)
    curr_token_type_id = speaker1_id
    for j in range(len(dialogues[i])):
        if j%2==0:
            curr_dial = curr_dial + " " + str(speaker1_id) + " " + str(dialogues[i][j])
            curr_token_type_id = curr_token_type_id + " ".join(len(str(dialogues[i][j]).split())*[speaker1_id])
        else:
            curr_dial = curr_dial + " " + str(speaker2_id) + " " + str(dialogues[i][j])
            curr_token_type_id = curr_token_type_id + " ".join(len(str(dialogues[i][j]).split())*[speaker2_id])


    curr_dial = curr_dial + " " + eos_id
    curr_token_type_id = curr_token_type_id + " " + speaker2_id


    list_d.append(curr_dial)
    list_t.append(curr_token_type_id)

  
    d = curr_dial.split()
    d.reverse()
    i = d.index("<Reply>")
    d.reverse()
    d[:-i] = [mask_id] * (len(d)-i)
    list_l.append(" ".join(d))


dialogue_csv['text'] = list_d
dialogue_csv['labels'] = list_l
dialogue_csv['token_type_ids'] = list_t



100%|██████████| 415951/415951 [00:14<00:00, 29682.48it/s]


In [ ]:
len(dialogue_csv)

415951

In [ ]:
train, test = train_test_split(dialogue_csv,test_size = 0.2)
train.to_csv("/content/drive/MyDrive/dialogues_train.csv",index = False)
test.to_csv("/content/drive/MyDrive/dialogues_test.csv",index = False)

In [ ]:
test.iloc[5]['text']

'<bos> <Question> Synchronised drone flying is a thing, flying in formation to make shapes etc, could it have just been that? <Reply> Oh that makes a lot of sense! I don’t have a lot of experience with drones and it was pretty dark, so it definitely could have been a group! Thanks! <|endoftext|>'

In [ ]:
# max_length = 768
# def tokenize_function(sample['text']):
#     token_type_id = []
#     labels = []

#         token_type_id.append([tokenizer(sample[i][0])] * len(sample[i].split()))
#         if (i==len(sample)-1):
#             labels.append(sample[i])
#         else:
#             labels.append([-100] * len(sample[i].split()))

#     token_type_id = token_type_id[:768]    
#     labels = labels[:768]
#     token = tokenizer(sample, padding=True, truncation=True, max_length=max_length)
#     token['token_type_ids'] = token_type_id
#     token['labels'] = labels
#     return token


In [ ]:
max_length = 128
def tokenize_function(sample):
    token = tokenizer(sample['text'], padding="max_length", return_attention_mask = False, truncation=True, max_length=max_length)
    token['token_type_ids'] = tokenizer(sample['token_type_ids'], padding="max_length", truncation=True, max_length=max_length)['input_ids']
    token['labels'] = tokenizer(sample['labels'], padding="max_length", truncation=True, max_length=max_length)['input_ids']
    return token


**Create Dataset Dictionary**

In [ ]:

dataset = load_dataset('/content/drive/MyDrive',
        data_files={
            'train': 'dialogues_train.csv',
            'test' : 'dialogues_test.csv'
        }
)

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/MyDrive-fafb74bbf1767846/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size = 1000,remove_columns=["text"])

  0%|          | 0/333 [00:00<?, ?ba/s]

  0%|          | 0/84 [00:00<?, ?ba/s]

In [ ]:
type(tokenized_dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
tokenized_dataset.save_to_disk('.')

In [ ]:
tokenizer.decode([10248, 10214, 13, 220])

'Good bot. '

**Initialize training arguments and Trainer API**

In [ ]:
training_args = TrainingArguments(
    r"/content/drive/MyDrive",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    optim="adafactor",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/checkpoint-12000")
## understand data collators
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    data_collator = data_collator,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

In [ ]:

# logger = logging.get_logger("transformers")

# # Setup logging
# logging.basicConfig(
#     format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
#     datefmt="%m/%d/%Y %H:%M:%S",
#     handlers=[logging.StreamHandler(sys.stdout)],
# )

# log_level = training_args.get_process_log_level()
# logger.setLevel(log_level)
# datasets.utils.logging.set_verbosity(log_level)
# transformers.utils.logging.set_verbosity(log_level)

logging.set_verbosity_info()
logger = logging.get_logger("transformers")
logger.info("INFO")

INFO


**Start Training**

In [ ]:
trainer.train("/content/drive/MyDrive/checkpoint-12000")

Loading model from /content/drive/MyDrive/checkpoint-12000.
***** Running training *****
  Num examples = 332760
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 15597
  Number of trainable parameters = 124443648
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 12000
  Will skip the first 2 epochs then the first 25632 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/25632 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
2,2.906800,2.770228


Saving model checkpoint to /content/drive/MyDrive/checkpoint-12500
Configuration saved in /content/drive/MyDrive/checkpoint-12500/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-12500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/checkpoint-13000
Configuration saved in /content/drive/MyDrive/checkpoint-13000/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-13000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/checkpoint-13500
Configuration saved in /content/drive/MyDrive/checkpoint-13500/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-13500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/checkpoint-14000
Configuration saved in /content/drive/MyDrive/checkpoint-14000/config.json
Model weights saved in /content/drive/MyDrive/checkpoint-14000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/checkpoint-14500
Configuration saved in /content/drive/MyD

TrainOutput(global_step=15597, training_loss=0.6709513565496871, metrics={'train_runtime': 9513.8182, 'train_samples_per_second': 104.929, 'train_steps_per_second': 1.639, 'total_flos': 6.6042606256128e+16, 'train_loss': 0.6709513565496871, 'epoch': 3.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_dataset["test"])

In [ ]:
trainer.save_model("/content/drive/MyDrive/final_model/")

Saving model checkpoint to /content/drive/MyDrive/
Configuration saved in /content/drive/MyDrive/config.json
Model weights saved in /content/drive/MyDrive/pytorch_model.bin


In [ ]:
device = torch.device("cuda")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

**Load Model**

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {
    'bos_token': "<bos>",
    'additional_special_tokens': ["<Question>", "<Reply>"],
    'pad_token': "<pad>",
    'mask_token' : "<msk>"
}

num_new_tokens = tokenizer.add_special_tokens(special_tokens)
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/checkpoint-15500")

**Execute Chatbot**

In [ ]:
sequence = ""
eos_id = tokenizer.encode("<|endoftext|>")
while(sequence.lower() not in ["quit","bye","goodbye","exit"]):
    sequence = input("User >> ")
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    if sequence.lower() in ["quit","bye","goodbye","exit"]:
        print("Watson >> Okay Goodbye!")
    else:
        final_outputs = model.generate(
                ids,
                do_sample=True,
                max_length=128,
                pad_token_id=model.config.eos_token_id,
                top_k=50,
                top_p=0.95,
            )
        answer = final_outputs[0]
        answer = tokenizer.decode(answer, skip_special_tokens=True)[len(sequence):]
        
        print("Watson >> " + answer)

User >> What happened to Tiffany Valiente?
Watson >> I can’t tell. I believe she was last seen in June of 2013. 

She had a cat and a rabbit on her front porch. Her mom just disappeared at that exact time. 

She is gone now and my friend may be back in the future. 

The police still have nothing to give but what evidence can I gather? 

Edit: She probably just dropped the cat off at a random time during the day.  

Edit 2: My old coworker said that she was last seen at a party at a nearby
User >> What happened to Peggy Mcguire
Watson >>  nanThis has been on the minds of all of us for the last few months and I wish I knew more. 
User >> quit
Watson >> Okay Goodbye!
